In [1]:
!pip install numba

In [2]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 35.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [3]:
from numba import jit
from ortools.linear_solver import pywraplp
import numpy as np

In [4]:
#Enter the sum of digits in your enrollment number as seed


def environment(seed):
  np.random.seed(seed)
  f = np.random.uniform(0.3,0.4,1)
  m = np.random.uniform(0.3,0.4,1)
  s = np.random.uniform(0.2,0.3,1)
  gamma = np.random.uniform(0.9,0.99,1)
  return f, m, s, gamma

seed = 15
f,m,s,gamma = environment(seed)
print('Transition probability parameter f:', f)
print('Transition probability parameter m:', m)
print('Transition probability parameter s:', s)
print('Discounting factor gamma:', gamma)


Transition probability parameter f: [0.38488177]
Transition probability parameter m: [0.31788959]
Transition probability parameter s: [0.20543632]
Discounting factor gamma: [0.93253846]


In [5]:
# def value_ite(f,m,s,gamma):
#   va = np.zeros(3)
#   vb = np.zeros(3)
#   action = np.zeros(2)
#   while(1):

#     #   0 - standing , 1 - moving , 2 - fallen

#     #   action 0 - slow - black ,   action 1 - aggressive - blue

#     # standing
#     action[0] = 1 + va[1]
#     action[1] = (1-s)*(2 + (gamma*va[1])) +  (s)*(-1 + (gamma*va[2]))
#     vb[0] = action.max()
#     # moving
#     action[0] = 1 + va[1]
#     action[1] = (m)*(-1 + (gamma*va[2])) + (1-m)*(2 + (gamma*va[1]))
#     vb[1] = action.max()

#     #fallen
#     vb[2] = (f)*(1 + (gamma*va[0])) + (1-f)*(-1 + (gamma*va[2]))

#     if(sum(vb)==sum(va)):
#       break
#     va=vb

#   return vb





In [27]:
F = f[0]
M = m[0]
S = s[0]
g = gamma[0]

# **Computing optimum value function**

In [63]:
va = np.zeros(3)
vb = np.zeros(3)
action = np.zeros(2)
while(1):
  #vb = np.zeros(3)*
  #   0 - standing , 1 - moving , 2 - fallen

  #   action 0 - slow,   action 1 - fast

  # standing
  action[0] = 1 + g*va[1]
  action[1] = (1-S)*(2 + (g*va[1])) +  (S)*(-1 + (g*va[2]))
  vb[0] = np.max(action)
  # moving
  action[0] = 1 + g*va[1]
  action[1] = (M)*(-1 + (g*va[2])) + (1-M)*(2 + (g*va[1]))
  vb[1] = np.max(action)

  #fallen
  vb[2] = (F)*(1 + (g*va[0])) + (1-F)*(-1 + (g*va[2]))

  if np.allclose(va,vb):
    break

  va=np.copy(vb)

In [64]:
v_star = vb

In [65]:
v_star

array([14.82169106, 14.82169106, 11.93637627])

In [66]:
print('Optimal value for standing state', v_star[0])
print('Optimal value for moving state', v_star[1])
print('Optimal value for fallen state', v_star[2])

Optimal value for standing state 14.82169106286883
Optimal value for moving state 14.82169106286883
Optimal value for fallen state 11.936376271000064


Thus optimal values are

---




*   **standing - 14.82169106286883**
*   **moving   - 14.82169106286883**
*   **fallen   - 11.936376271000064**

# **Optimal action values**

In [10]:
# def q_star(f,m,s,gamma,v):
#   Q = np.zeros([3,2])

#   T = np.zeros([3,2,3])
#   T[0][0]= [0,1,0]
#   T[0][1]= [0,1-s[0],s[0]]
#   T[1][0]= [0,1,0]
#   T[1][1]= [0,1-m[0],m[0]]
#   T[2][0]= [f[0],0,1-f[0]]
#   T[2][1]= [0,0,0]

#   R = np.zeros([3,2,3])
#   R[0][0]= [0,1,0]
#   R[0][1]= [0,2,-1]
#   R[1][0]= [0,1,0]
#   R[1][1]= [0,2,-1]
#   R[2][0]= [1,0,-1]
#   R[2][1]= [0,0,0]

#   for i in range(3):
#     for j in range(2):
#       Q[i][j] = sum(T[i][j]*(R[i][j] + gamma[0]*v))
#   return Q


In [67]:
Q_star = np.zeros([3,2])

T = np.zeros([3,2,3])
T[0][0]= [0,1,0]
T[0][1]= [0,1-s[0],s[0]]
T[1][0]= [0,1,0]
T[1][1]= [0,1-m[0],m[0]]
T[2][0]= [f[0],0,1-f[0]]
T[2][1]= [0,0,0]

R = np.zeros([3,2,3])
R[0][0]= [0,1,0]
R[0][1]= [0,2,-1]
R[1][0]= [0,1,0]
R[1][1]= [0,2,-1]
R[2][0]= [1,0,-1]
R[2][1]= [0,0,0]

for i in range(3):
  for j in range(2):
    Q_star[i][j] = sum(T[i][j]*(R[i][j] + gamma[0]*v_star))

In [68]:
Q_star

array([[14.82179696, 14.65272726],
       [14.82179696, 14.01279314],
       [11.93648217,  0.        ]])

In [69]:
print('Q*(standing,slow)       = ', Q_star[0][0])
print('Q*(standing,aggressive) = ', Q_star[0][1])
print('Q*(moving,slow)         = ', Q_star[1][0])
print('Q*(moving,aggressive)   = ', Q_star[1][1])
print('Q*(fallen,slow)         = ', Q_star[2][0])

Q*(standing,slow)       =  14.821796960556183
Q*(standing,aggressive) =  14.652727262908204
Q*(moving,slow)         =  14.821796960556183
Q*(moving,aggressive)   =  14.012793142769047
Q*(fallen,slow)         =  11.936482168687418


Thus optimal policy is

---




*   **standing - slow action**
*   **moving   - slow action**
*   **fallen   - slow action(default)**






In [76]:
va = np.zeros(3)
vb = np.zeros(3)
while(1):

  vb[0] = sum(T[0][0]*(R[0][0] + gamma*va))
  vb[1] = sum((T[1][0])*(R[1][0] + gamma*va))
  vb[2] = sum((T[2][0])*(R[2][0] + gamma*va))

  if(vb[0]==va[0]):
    if(vb[1]==va[1]):
      if(vb[2]==va[2]):
        break

  va=np.copy(vb)

In [77]:
v_pi_star = vb

In [81]:
v_pi_star[2]

11.937946020039078

Thus optimal values obtained from pi* are

---




*   **standing - 14.823260811907842**
*   **moving   - 14.823260811907842**
*   **fallen   - 11.937946020039078**

# **Using OR tools**

In [18]:
solver = pywraplp.Solver.CreateSolver("GLOP")

In [19]:
x = solver.NumVar(0, solver.infinity(), "x")
y = solver.NumVar(0, solver.infinity(), "y")
z = solver.NumVar(0, solver.infinity(), "z")

In [20]:
solver.constraints()

[]

In [21]:
#constraints - standing
solver.Add(x - g*y >=1)
solver.Add(x - (1-S) * g * y - S * g * z >= 2 - 3*S)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x79ae7eb6e580> >

In [22]:
#constraints - moving
solver.Add((1-g) * y >= 1)
solver.Add((1 - g*(1-M)) * y - M*g * z >= 2 - 3*M)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x79ae7eb6e670> >

In [23]:
#constraint - fallen
solver.Add( -F*g * x + (1-(1-F)*g) * z >= 2*F - 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x79ae7eb6e5b0> >

In [24]:
#objective function
solver.Minimize(x + y +z)

In [25]:
results = solver.Solve()

In [26]:
#results
if results == pywraplp.Solver.OPTIMAL:
        print('Objective value =', solver.Objective().Value())
        print('optimal value of standing: ', x.solution_value())
        print('optimal value of moving:   ', y.solution_value())
        print('optimal value of fallen:   ', z.solution_value())
else:
        print('The problem does not have an optimal solution.')

Objective value = 41.5844676438548
optimal value of standing:  14.823260811907854
optimal value of moving:    14.823260811907854
optimal value of fallen:    11.937946020039092


# **Policy Iteration**

In [86]:
# #starting with arbitrary policy
# policy = np.zeros(3)
# for i in range(3):
#     policy[i] = (np.random.randint(0,2))

*Howards policy iteration*

In [ ]:
# while(1):

#   #standing
#   IS = np.zeros(3)


In [89]:
def howard_policy_iteration(T, R, n_actions, n_states, gamma, max_iterations=1000, tolerance=1e-6):
    # Initialize the value function and policy
    V = np.zeros(n_states)
    policy = np.zeros((n_states, n_actions))

    for _ in range(max_iterations):
        delta = 0
        for s in range(n_states):
            v = V[s]
            for a in range(n_actions):
                new_v = sum(R[s][a] + gamma * np.sum(T[s][a] * V))
                policy[s][a] = new_v
            V[s] = np.max(policy[s])
            delta = max(delta, abs(v - V[s]))
        if delta < tolerance:
            break

    # Extract the optimal policy
    optimal_policy = np.zeros(n_states, dtype=int)
    for s in range(n_states):
        optimal_policy[s] = np.argmax(policy[s])

    return optimal_policy.reshape(-1, 1)

In [90]:
n_actions = 2
n_states = 3

optimal_policy = howard_policy_iteration(T, R, n_actions, n_states, gamma)
print("Optimal Policy:")
print(optimal_policy)

Optimal Policy:
[[0]
 [0]
 [0]]


<ipython-input-89-13d54e38ca73>:11: RuntimeWarning: overflow encountered in scalar add
  new_v = sum(R[s][a] + gamma * np.sum(T[s][a] * V))
<ipython-input-89-13d54e38ca73>:11: RuntimeWarning: invalid value encountered in multiply
  new_v = sum(R[s][a] + gamma * np.sum(T[s][a] * V))


*Simple policy iteration*

In [91]:
def simple_policy_iteration(T, R, n_actions, n_states, gamma, max_iterations=1000, tolerance=1e-6):
    # Initialize the value function and policy
    V = np.zeros(n_states)
    policy = np.zeros((n_states, n_actions))

    for _ in range(max_iterations):
        # Policy Evaluation
        for s in range(n_states):
            v = V[s]
            action_values = np.zeros(n_actions)
            for a in range(n_actions):
                action_values[a] = sum(R[s][a] + gamma * np.sum(T[s][a] * V))
            policy[s] = np.argmax(action_values)
            V[s] = np.max(action_values)

        # Policy Improvement
        policy_stable = True
        for s in range(n_states):
            old_action = np.argmax(policy[s])
            action_values = np.zeros(n_actions)
            for a in range(n_actions):
                action_values[a] = sum(R[s][a] + gamma * np.sum(T[s][a] * V))
            policy[s] = np.argmax(action_values)
            if old_action != np.argmax(action_values):
                policy_stable = False

        if policy_stable:
            break

    # Extract the optimal policy
    optimal_policy = np.zeros(n_states, dtype=int)
    for s in range(n_states):
        optimal_policy[s] = np.argmax(policy[s])

    return optimal_policy.reshape(-1, 1)


In [92]:
n_actions = 2
n_states = 3

optimal_policy = simple_policy_iteration(T, R, n_actions, n_states, gamma)
print("Optimal Policy:")
print(optimal_policy)

Optimal Policy:
[[0]
 [0]
 [0]]


<ipython-input-91-0c3969d7422e>:12: RuntimeWarning: overflow encountered in scalar add
  action_values[a] = sum(R[s][a] + gamma * np.sum(T[s][a] * V))
<ipython-input-91-0c3969d7422e>:22: RuntimeWarning: invalid value encountered in multiply
  action_values[a] = sum(R[s][a] + gamma * np.sum(T[s][a] * V))
<ipython-input-91-0c3969d7422e>:12: RuntimeWarning: invalid value encountered in multiply
  action_values[a] = sum(R[s][a] + gamma * np.sum(T[s][a] * V))


In [ ]:
# va = np.zeros(3)
# vb = np.zeros(3)
# while(1):

#   vb[0] = (1-s)*(2 + (gamma*va[1])) +  (s)*(-1 + (gamma*va[2]))
#   vb[1] = 1 + va[1]
#   vb[2] = (f)*(1 + (gamma*va[0])) + (1-f)*(-1 + (gamma*va[2]))

#   if(vb[0]==va[0]):
#     if(vb[1]==va[1]):
#       if(vb[2]==va[2]):
#         break

#   va=vb

In [ ]:
#vb